In [1]:
import os, io, json, urllib, numpy as np, codecs
import pandas as pd

In [2]:
    url="https://api.crossref.org/journals?rows=1"
    response = urllib.urlopen(url).read()
    json_result=json.loads(response)
    count=json_result['message']['total-results']

In [3]:
base_url="https://api.crossref.org/journals?rows=1000&offset="

In [4]:
#f=codecs.open('journals.csv','w','utf-8')
dic=[]
for it in range(count/1000): # change
    print it,
    offset = it * 1000
    url=base_url+str(offset)
    response = urllib.urlopen(url).read()
    json_result=json.loads(response)
    items=json_result['message']['items']
    for item in items:
        if len(item['ISSN'])>0 and 'title' in item and 'publisher' in item: 
            journal={}
            journal['e:title']=item['title'].strip()
            journal['e:publisher']=item['publisher'] 
            journal['e:issn']=item['ISSN']
            journal['e:subjects']=[]
            for subject in item['subjects']:
                journal['e:subjects'].append(subject['name'])
            if journal not in dic:
                dic.append(journal)
dic_c=dic      

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52


In [5]:
unique_subjects=[]
for j in dic:
    for s in j['e:subjects']: 
        if s not in unique_subjects: 
            unique_subjects.append(s)
len(unique_subjects)            

330

In [6]:
pd.DataFrame(unique_subjects).to_csv('d/unique_subjects.csv',header=False,index=False,encoding='utf-8')

In [7]:
df=pd.DataFrame(dic)
df.to_csv('d/journals.csv',index=False,encoding='utf8')

In [8]:
main_issns=pd.read_csv('main_issns.csv',header=None)
main_issns=main_issns[0].values

In [9]:
dic_new=[]
for j in dic:
    j_id=j
    if j['e:issn'][0] in main_issns:        
        j_id['@id']=j['e:issn'][0]
        dic_new.append(j_id)    
    if len(j['e:issn'])>1:
        if j['e:issn'][1] in main_issns:
            j_id['@id']=j['e:issn'][1]
            dic_new.append(j_id)    
len(dic_new)        

44972

In [11]:
with open('d/road.tsv') as f:
    road_lines = f.read().splitlines()
road_lines=road_lines[1:]

In [12]:
road_urls=[issn[1:-1] for issn in road_lines]
road_issns=[url[-9:] for url in road_urls]

In [13]:
i=0
for journal in dic_new:
    issn=journal['@id']
    if journal['@id'] in road_issns:
        journal['e:open']=1
        journal['e:road_url']=road_urls[road_issns.index(issn)]
        dic_new[i]=journal
    i+=1

In [14]:
batches=len(dic_new)/10000
for s in range(batches+1):
    dic_chunk=dic_new[10000*s:10000*(s+1)]
    with open('jsonld/'+str(s)+'journals.jsonld', 'w') as outfile:
        json.dump(dic_chunk, outfile)